In [21]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [22]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai.embeddings import OpenAIEmbeddings

In [23]:
# 문자열 리스트로 생성
db = FAISS.from_texts(
    ["제 이름은 이인환입니다.", "저는 컴퓨터프로그래밍을 교육하는 강사입니다."],
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    metadatas=[{"source": "텍스트문서"}, {"source": "텍스트문서"}],
    ids=["doc1", "doc2"],
)

In [24]:
# 저장된 결과를 확인합니다. id 값은 지정한 id 값이 잘 들어가 있는지 확인합니다.
# 저장된 내용
db.docstore._dict

{'doc1': Document(metadata={'source': '텍스트문서'}, page_content='제 이름은 이인환입니다.'),
 'doc2': Document(metadata={'source': '텍스트문서'}, page_content='저는 컴퓨터프로그래밍을 교육하는 강사입니다.')}

In [26]:
# 텍스트로부터 추가 (add_texts)

In [29]:
# add_texts() 텍스트를 임베딩하고 벡터 저장소에 추가하는 기능을 제공합니다.

# 매개변수
# 
# . texts (Iterable[str])               : 벡터 저장소에 추가할 텍스트 이터러블
# . metadatas (Optional[List[dict]])    : 텍스트와 연관된 메타데이터 리스트 (선택적)
# . ids (Optional[List[str]])           : 텍스트의 고유 식별자 리스트 (선택적)
# . **kwargs                            : 추가 키워드 인자

In [30]:
# 데이터 추가
db.add_texts(
    ["이번엔 텍스트 데이터를 추가합니다.", "추가한 2번째 텍스트 데이터 입니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["new_doc2", "new_doc3"],
)

['new_doc2', 'new_doc3']

In [32]:
# 데이터 아이디
db.index_to_docstore_id

{0: 'doc1', 1: 'doc2', 2: 'new_doc2', 3: 'new_doc3'}

In [33]:
# 문서 삭제 (Delete Documents)

In [35]:
# delete() 벡터 저장소에서 지정된 ID에 해당하는 문서를 삭제하는 기능을 제공합니다.

# 매개변수
# . ids (Optional[List[str]]): 삭제할 문서의 ID 리스트
# . **kwargs: 추가 키워드 인자 (이 메서드에서는 사용되지 않음)

In [36]:
# 삭제할 데이터를 추가
ids = db.add_texts(
    ["삭제용 데이터를 추가합니다.", "2번째 삭제용 데이터입니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["delete_doc1", "delete_doc2"],
)

In [37]:
# 삭제할 id 를 확인
print(ids)

['delete_doc1', 'delete_doc2']


In [38]:
# id 로 삭제
db.delete(ids)

True

In [40]:
# 삭제된 결과를 출력
db.index_to_docstore_id

{0: 'doc1', 1: 'doc2', 2: 'new_doc2', 3: 'new_doc3'}

In [43]:
# 저장
db.save_local(folder_path="./faiss_db", index_name="faiss_index")

In [44]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="./faiss_db",
    index_name="faiss_index",
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True,
)

In [45]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: 'doc1', 1: 'doc2', 2: 'new_doc2', 3: 'new_doc3'}

In [46]:
# 검색기로 변환
retriever = loaded_db.as_retriever()

In [47]:
retriever.invoke('컴퓨터프로그래밍?')

[Document(metadata={'source': '텍스트문서'}, page_content='저는 컴퓨터프로그래밍을 교육하는 강사입니다.'),
 Document(metadata={'source': 'mydata.txt'}, page_content='이번엔 텍스트 데이터를 추가합니다.'),
 Document(metadata={'source': 'mydata.txt'}, page_content='추가한 2번째 텍스트 데이터 입니다.'),
 Document(metadata={'source': '텍스트문서'}, page_content='제 이름은 이인환입니다.')]